In [1]:
import cv2
import numpy as np
import time

# Load Yolo
net = cv2.dnn.readNet('G:\Mitangshu_TensorFlow\Mask_Yolo\Mask_NOMask\yolov3_training_last.weights', 'G:\Mitangshu_TensorFlow\Mask_Yolo\yolov3_testing_mask.cfg')
classes = []
with open("/Mitangshu_Tensorflow/Mask_Yolo/coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]
layer_names = net.getLayerNames()
output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
colors = np.random.uniform(0, 255, size=(len(classes), 3))

# Loading camera
cap = cv2.VideoCapture(0)

font = cv2.FONT_HERSHEY_PLAIN
starting_time = time.time()
frame_id = 0

#while loop to extract frame from the camera
while True:
    _, frame = cap.read()
    frame_id += 1

    height, width, channels = frame.shape
    
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (320, 320), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)
    # Showing informations on the screen
    class_ids = []
    confidences = []
    boxes = []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                # Object detected
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                # Rectangle coordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.4, 0.3)
    
    elapsed_time = time.time() - starting_time
    fps = frame_id / elapsed_time
    cv2.rectangle(frame,(0,0), (120, 150), (204,221,136), thickness = cv2.FILLED )
    cv2.rectangle(frame,(0,0), (320, 40), (170,198,57), thickness = cv2.FILLED )

    
    cv2.putText(frame, "Mask Detector 1.0", (5, 30), font, 2, (0,0,0), 2)
    cv2.putText(frame, "FPS: " + str(round(fps, 2)), (5, 60), font, 1, (0, 0, 0), 1)
    cv2.putText(frame, "Mask= " ,(5, 85), font, 1, (0,0,0), 1 )
    cv2.putText(frame, "No Mask= ",(5,110), font, 1, (0,0,0), 1 )
    mask_counter = 0
    nomask_counter = 0

    for i in range(len(boxes)):

        if i in indexes:
            
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            confidence = confidences[i]
            color = colors[class_ids[i]]
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
            cv2.rectangle(frame, (x, y), (x + w, y + 30), color, -1)
            cv2.putText(frame, label + " " + str(round(confidence, 2)), (x, y + 30), font, 1.5, (255,255,255), 2)
            #if confidence>0.6:
                #cv2.putText(frame,"Boxes: "+ str(i),(5, 120), font, 1, (0,0,0), 1 )
            if class_ids[i] == 0:
                mask_counter =mask_counter+ 1

            
            elif class_ids[i] == 1:
                nomask_counter =nomask_counter+ 1
            
    
    safe = (mask_counter-nomask_counter)
    if safe<0:
        
        cv2.rectangle(frame,(5,120),(110, 140), (0,0,255), thickness = cv2.FILLED)
        cv2.putText(frame,"UNSAFE",(15,135),font,1.5,(255,255,255),1)
    else:
        cv2.rectangle(frame,(5,120),(110, 140), (0,255,0), thickness = cv2.FILLED)
        cv2.putText(frame,"SAFE",(15,135),font,1.5,(255,255,255),1)
    
        
    cv2.putText(frame,str(mask_counter),(70, 85), font, 1, (0,0,0), 1 )
    cv2.putText(frame,str(nomask_counter),(95,110), font, 1, (0,0,0), 1 )

               
    cv2.imshow("Image", frame)
    
    key = cv2.waitKey(1)
    if key == 88:
        break

cap.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 

In [3]:
# Name custom object
import cv2
import numpy as np
import time
import glob
import random


classes = ['Mask', "No Mask"]
def Image_detect(image_path):
# Images path
    images_path = glob.glob(image_path)
    

    layer_names = net.getLayerNames()
    output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
    colors = np.random.uniform(0, 255, size=(len(classes), 3))
    print(len(layer_names))
    # Insert here the path of your images
    random.shuffle(images_path)
    # loop through all the images
    for img_path in images_path:
        # Loading image
        img = cv2.imread(img_path)
        img = cv2.resize(img, (480,480), fx=0.4, fy=0.4)
        height, width, channels = img.shape

        # Detecting objects
        blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)

        net.setInput(blob)
        outs = net.forward(output_layers)

            # Showing informations on the screen
        class_ids = []
        confidences = []
        boxes = []
        for out in outs:
            for detection in out:

                scores = detection[5:]
                class_id = np.argmax(scores)

                confidence = scores[class_id]
                if confidence > 0.4:
                    # Object detected

                    center_x = int(detection[0] * width)
                    center_y = int(detection[1] * height)
                    w = int(detection[2] * width)
                    h = int(detection[3] * height)

                    # Rectangle coordinates
                    x = int(center_x - w / 2)
                    y = int(center_y - h / 2)

                    boxes.append([x, y, w, h])
                    confidences.append(float(confidence))
                    class_ids.append(class_id)

        indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
        #print(x,y,w,h)
        #print(scores)

        font = cv2.FONT_HERSHEY_PLAIN
        
        
        cv2.rectangle(img,(0,0), (120, 150), (204,221,136), thickness = cv2.FILLED )
        cv2.rectangle(img,(0,0), (320, 40), (170,198,57), thickness = cv2.FILLED )

    
        cv2.putText(img, "Mask Detector 1.0", (5, 30), font, 2, (0,0,0), 2)
        cv2.putText(img, "FPS: " + str(round(fps, 2)), (5, 60), font, 1, (0, 0, 0), 1)
        cv2.putText(img, "Mask= " ,(5, 85), font, 1, (0,0,0), 1 )
        cv2.putText(img, "No Mask= ",(5,110), font, 1, (0,0,0), 1 )
        mask_counter = 0
        nomask_counter = 0
        for i in range(len(boxes)):
            if i in indexes:

                x, y, w, h = boxes[i]
                label = str(classes[class_ids[i]])
                confidence = confidences[i]
                color = colors[class_ids[i]]
                cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
                cv2.rectangle(img, (x, y), (x + w, y + 30), color, -1)
                cv2.putText(img, label + " " + str(round(confidence, 2)), (x, y + 30), font, 1.5, (255,255,255), 2)
                
                if class_ids[i] == 0:
                    mask_counter =mask_counter+ 1

            
                elif class_ids[i] == 1:
                    nomask_counter =nomask_counter+ 1
            
    
        safe = (mask_counter-nomask_counter)
        if safe<0:
        
            cv2.rectangle(img,(5,120),(110, 140), (0,0,255), thickness = cv2.FILLED)
            cv2.putText(img,"UNSAFE",(15,135),font,1.5,(255,255,255),1)
        else:
            cv2.rectangle(img,(5,120),(110, 140), (0,255,0), thickness = cv2.FILLED)
            cv2.putText(img,"SAFE",(15,135),font,1.5,(255,255,255),1)
    
        
        cv2.putText(img,str(mask_counter),(70, 85), font, 1, (0,0,0), 1 )
        cv2.putText(img,str(nomask_counter),(95,110), font, 1, (0,0,0), 1 )



        cv2.imshow("Image", img)
        key = cv2.waitKey(0)

    cv2.destroyAllWindows()

Image_detect("G:\StudyMaterial5thSEM\Mini project\*.jpg")

254


KeyboardInterrupt: 